In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
from pandas.plotting import register_matplotlib_converters
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
register_matplotlib_converters()
from time import time

In [2]:
def parser(s):
    return datetime.strptime(s, '%Y-%m-%d')

In [3]:
#read data

df=pd.read_csv("C:/Users/travi/Documents/steps_p01.csv", parse_dates=[1], index_col=1, squeeze=True, date_parser=parser)

In [4]:
df.head()

,Participant_ID,Steps
Dates,,
2019-11-01,p01,4664
2019-11-02,p01,3035
2019-11-04,p01,1284
2019-11-05,p01,4966
2019-11-08,p01,2094


In [5]:
df.dtypes

Participant_ID    object
Steps              int64
dtype: object

In [6]:
df_copy=df.drop(labels = ["Participant_ID"], axis=1)

In [7]:
df_copy.head()


,Steps
Dates,
2019-11-01,4664
2019-11-02,3035
2019-11-04,1284
2019-11-05,4966
2019-11-08,2094


In [8]:
df_copy.dtypes

Steps    int64
dtype: object

In [9]:
df_copy.count()

Steps    117
dtype: int64

In [10]:

# grid search ARIMA parameters for time series
import warnings
from math import sqrt
from pandas import read_csv
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

In [11]:
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
	# prepare training dataset
	train_size = int(len(X) * 0.66)
	train, test = X[0:train_size], X[train_size:]
	history = [x for x in train]
	# make predictions
	predictions = list()
	for t in range(len(test)):
		model = ARIMA(history, order=arima_order)
		model_fit = model.fit()
		yhat = model_fit.forecast()[0]
		predictions.append(yhat)
		history.append(test[t])
	# calculate out of sample error
	#rmse = sqrt(mean_squared_error(test, predictions))
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
	return rmse

In [12]:
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)
				try:
					rmse = evaluate_arima_model(dataset, order)
					if rmse < best_score:
						best_score, best_cfg = rmse, order
					print('ARIMA%s RMSE=%.3f' % (order,rmse))
				except:
					continue
	print('Best ARIMA%s RMSE=%.3f' % (best_cfg, best_score))
 

In [14]:
# evaluate parameters
p_values = [0, 1, 2, 4, 6, 8, 10]
d_values = range(0, 3)
q_values = range(0, 3)
warnings.filterwarnings("ignore")
evaluate_models(df_copy.values, p_values, d_values, q_values)

ARIMA(0, 0, 0) RMSE=3088.720
ARIMA(0, 0, 1) RMSE=3120.266
ARIMA(0, 0, 2) RMSE=3084.247
ARIMA(0, 1, 0) RMSE=4429.597
ARIMA(0, 1, 1) RMSE=3083.468
ARIMA(0, 1, 2) RMSE=3115.521
ARIMA(0, 2, 0) RMSE=7421.318
ARIMA(0, 2, 1) RMSE=4452.821
ARIMA(0, 2, 2) RMSE=3156.881
ARIMA(1, 0, 0) RMSE=3117.978
ARIMA(1, 0, 1) RMSE=3113.238
ARIMA(1, 0, 2) RMSE=3222.499
ARIMA(1, 1, 0) RMSE=4100.564
ARIMA(1, 1, 1) RMSE=3115.089
ARIMA(1, 1, 2) RMSE=3120.028
ARIMA(1, 2, 0) RMSE=6098.486
ARIMA(1, 2, 1) RMSE=4120.338
ARIMA(1, 2, 2) RMSE=3135.232
ARIMA(2, 0, 0) RMSE=3095.753
ARIMA(2, 0, 1) RMSE=3121.030
ARIMA(2, 0, 2) RMSE=3160.873
ARIMA(2, 1, 0) RMSE=3855.541
ARIMA(2, 1, 1) RMSE=3103.523
ARIMA(2, 1, 2) RMSE=3202.717
ARIMA(2, 2, 0) RMSE=5275.415
ARIMA(2, 2, 1) RMSE=3875.646
ARIMA(2, 2, 2) RMSE=3536.237
ARIMA(4, 0, 0) RMSE=3043.447
ARIMA(4, 0, 1) RMSE=3303.827
ARIMA(4, 0, 2) RMSE=3207.444
ARIMA(4, 1, 0) RMSE=3483.102
ARIMA(4, 1, 1) RMSE=3067.731
ARIMA(4, 1, 2) RMSE=3141.007
ARIMA(4, 2, 0) RMSE=4743.675
ARIMA(4, 2, 1)

In [ ]:
# https://machinelearningmastery.com/grid-search-arima-hyperparameters-with-python/
# https://github.com/ritvikmath/Time-Series-Analysis/blob/master/SARIMA%20Model.ipynb
# https://www.youtube.com/watch?v=Al8m6K_stfA
# https://towardsdatascience.com/most-useful-python-functions-for-time-series-analysis-ed1a9cb3aa8b
# https://www.statsmodels.org/dev/examples/notebooks/generated/stationarity_detrending_adf_kpss.html
